In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse, parse_qs
from selenium import webdriver
import time as t
from tqdm import tqdm
# disable warnings
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
vtc_url = "https://vtcnews.vn/"
sub_page_vtc=['thoi-su-28', 'kinh-te-29', 'the-thao-34', 'the-gioi-30',
              'giao-duc-31', 'phap-luat-32', 'giai-tri-33', 'suc-khoe-35', 
              'khoa-hoc-cong-nghe-82', 'oto-xe-may-37', 
              'phong-su-kham-pha-36', 'song-ket-noi-123']
vtc_news = [vtc_url + i for i in sub_page_vtc]

lao_dong_url = "https://laodong.vn/"
sub_page_ld = ["xa-hoi", "moi-truong", "giao-thong", "kinh-doanh", 
         "thoi-su", "the-gioi", "phap-luat", "tu-van-phap-luat", 
         "an-ninh-hinh-su", "the-thao", "bong-da", "bong-da-quoc-te",
         "lich-thi-dau", "golf", "tennis", "y-te", "tlv-canh-doi",
         "dinh-duong-am-thuc", "lam-dep", "cac-loai-benh", "van-hoa",
         "giai-tri", "thoi-trang", "sach-hay", "cach-lam-hay-tu-co-so",
         "vi-loi-ich-doan-vien", "the-gioi-so", "vu-khi", "quan-su",
         "nha-dep", "quy-hoach", "thi-truong-xe", "lai-xe-an-toan",
         "tu-van-lao-dong", "xuat-khau-lao-dong", "chinh-sach-giao-duc",
         "tuyen-sinh", "chuyen-nha-minh", "yeu-360", "nuoi-con", 
         "tlv-tin-hoat-dong"]
ld_news = [lao_dong_url + i for i in sub_page_ld]

tuoi_tre_url = "https://tuoitre.vn/"
sub_page_tt = ["thoi-su", "phap-luat", "kinh-doanh", "the-gioi", "van-hoa", "giao-duc", "suc-khoe",
         "du-lich", "the-thao", "cong-nghe", "giai-tri", "xe", "nhip-song-tre", "nha-dat", "gia-that", "ban-doc"]
tt_news = [tuoi_tre_url + i for i in sub_page_tt]

dantri_url = "https://dantri.com.vn/"
sub_page_dt = ["xa-hoi", "kinh-doanh", "the-gioi", "giai-tri", "bat-dong-san", "the-thao",
               "viec-lam", "suc-khoe", "o-to-xe-may", "suc-manh-so", "giao-duc", "an-sinh",
               "phap-luat", "du-lich", "doi-song", "tinh-yeu-gioi-tinh", "khoa-hoc-cong-nghe"]
dt_news = [dantri_url + i for i in sub_page_dt]

vnexpress_url = "https://vnexpress.net/"
sub_page_vne = ["thoi-su", "the-gioi", "kinh-doanh", "giai-tri", "the-thao", 
                "phap-luat", "giao-duc", "suc-khoe", "doi-song", "du-lich", 
                "khoa-hoc", "so-hoa", "oto-xe-may", "bat-dong-san", "y-kien"]
vne_news = [vnexpress_url + i for i in sub_page_vne]



In [3]:
dict_topic = {"thoi-su": "Thời sự", "kinh-te": "Kinh tế", "the-thao": "Thể thao", "the-gioi": "Thế giới",
                "giao-duc": "Giáo dục", "phap-luat": "Pháp luật", "giai-tri": "Giải trí", "suc-khoe": "Sức khỏe",
                "khoa-hoc-cong-nghe": "Khoa học công nghệ", "oto-xe-may": "Ô tô xe máy", "phong-su-kham-pha": "Phóng sự khám phá",
                "song-ket-noi": "Sống kết nối", "xa-hoi": "Xã hội", "moi-truong": "Môi trường", "giao-thong": "Giao thông",
                "dinh-duong-am-thuc": "Dinh dưỡng ẩm thực", "lam-dep": "Làm đẹp", "cac-loai-benh": "Các loại bệnh",
                "van-hoa": "Văn hóa", "thoi-trang": "Thời trang", "sach-hay": "Sách hay", "cach-lam-hay-tu-co-so": "Cách làm hay từ cơ sở",
                "vi-loi-ich-doan-vien": "Vì lợi ích đoàn viên", "the-gioi-so": "Thế giới số", "vu-khi": "Vũ khí", "quan-su": "Quân sự",
                "nha-dep": "Nhà đẹp", "quy-hoach": "Quy hoạch", "thi-truong-xe": "Thị trường xe", "lai-xe-an-toan": "Lái xe an toàn",
                "tu-van-lao-dong": "Tư vấn lao động", "xuat-khau-lao-dong": "Xuất khẩu lao động", "chinh-sach-giao-duc": "Chính sách giáo dục",
                "tuyen-sinh": "Tuyển sinh", "chuyen-nha-minh": "Chuyện nhà mình", "yeu-360": "Yêu 360", "nuoi-con": "Nuôi con",
                "tlv-canh-doi": "TLV cảnh đời", "tlv-tin-hoat-dong": "TLV tin hoạt động", "bong-da": "Bóng đá",
                "bong-da-quoc-te": "Bóng đá quốc tế", "lich-thi-dau": "Lịch thi đấu", "golf": "Golf", "tennis": "Tennis",
                "y-te": "Y tế", "an-ninh-hinh-su": "An ninh hình sự", "tu-van-phap-luat": "Tư vấn pháp luật", "nhip-song-tre": "Nhịp sống trẻ",
                "nha-dat": "Nhà đất", "gia-that": "Giá thất", "ban-doc": "Bạn đọc", "viec-lam": "Việc làm", "suc-manh-so": "Sức mạnh số",
                "tinh-yeu-gioi-tinh": "Tình yêu giới tính", "doi-song": "Đời sống", "bat-dong-san": "Bất động sản", "an-sinh": "An sinh",
                "kinh-doanh": "Kinh doanh", "phap-luat": "Pháp luật", "du-lich": "Du lịch", "cong-nghe": "Công nghệ", "xe": "Xe", "y-kien": "Ý kiến",
                "so-hoa": "Số hóa", "khoa-hoc": "Khoa học", "o-to-xe-may": "Ô tô xe máy", "thi-truong": "Thị trường"}

In [4]:
data4 = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic", "time", "imglink"])

In [15]:
def find_all_articles_at_vtc(max_sl, url, data4):
    articles_list = []
    vtc_news_pages = []
    vtc_news_pages.append(url)
    for i in range(2, max_sl):
        vtc_news_pages.append(f"{url}/trang-{i}.html")


    def get_vtc_articles(weburl, data):
        response = requests.get(weburl,verify=False)
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Combine both types of article containers in one loop
        article_containers = soup.find_all('article', class_=['clearfix distance', 'distance clearfix'])
        
        for article in article_containers:
            title_element = article.select_one('h3.title-1 a')
            abstract_element = article.select_one('p')
            img_element = article.select_one('figure img')
            time_element = article.select_one('footer span.time-update')
            
            if title_element and abstract_element and img_element and time_element:
                title = title_element.get('title', 'No Title')
                link = title_element.get('href', '#')
                abstract = abstract_element.get_text(strip=True)
                imglink = img_element.get('data-src', '')
                time = time_element.text.split(' ')
                topic = url.split('/')[-1]
                topic = topic[:topic.rfind("-")]
                
                if len(time) >= 2:
                    time_formatted = f"{time[1]} {time[0]}"
                else:
                    time_formatted = "Unknown Time"
                
                # Store each article information in a dictionary and add it to the list
                articles_list.append({
                    "title": title,
                    "abstract": abstract,
                    "source": "Báo VTC",
                    "link": f"https://vtcnews.vn{link}",
                    "topic": dict_topic[topic],
                    "time": time_formatted,
                    "imglink": imglink
                })
        
        return pd.concat([data, pd.DataFrame(articles_list)], ignore_index=True)

    for page in vtc_news_pages:
        data4 = get_vtc_articles(page, data4)
    
    return data4


def find_all_articles_at_ld(max_sl, url, data4):
    ld_new_pages = []
    for i in range(1, max_sl):
        ld_new_pages.append(f"{url}?page={i}")

    def get_ld_articles(url):
        response = requests.get(url,verify=False)

        soup = BeautifulSoup(response.content, 'html.parser')
        articles_list = []
        article_containers = soup.find_all(name = 'article')

        for article in article_containers:
            title_element = article.select_one('h2',class_="title")
            link_title_element = article.select_one("a", class_="link-title")
            abstract_element = article.select_one(".chapeau")
            img_element = article.select_one('figure img')
            time_element = article.select_one('span.time')
            
            if title_element and abstract_element and img_element and time_element:
                title = title_element.text
                link = link_title_element.get('href')
                abstract = abstract_element.text
                imglink = img_element.get('data-src')
                time = time_element.text.strip().split(" ")
                topic = url.split('/')[-1]
                topic = topic[:topic.find("?")]

                # Format the time into a readable format
                if time and len(time) >= 2:
                    time_formatted = f"{time[0]} {time[1]}"
                else:
                    time_formatted = "Unknown Time"

                # Append the article's information to the list
                articles_list.append({
                    "title": title,
                    "abstract": abstract,
                    "source": "Báo Lao Động",
                    "link": f"https://laodong.vn/{link}",
                    "topic": dict_topic[topic],
                    "time": time_formatted,
                    "imglink": imglink
                })
        return articles_list
    
    for page in ld_new_pages:
        data4 = pd.concat([data4, pd.DataFrame(get_ld_articles(page))], ignore_index=True)

    return data4



def find_all_articles_at_tt(url, data4):
    browser = webdriver.Chrome()
    browser.get(url)
    screen_height = browser.execute_script("return window.screen.height;") * 2 # Browser window height times 3  
    i = 1
    while True:
        browser.execute_script(f"window.scrollTo(0, {screen_height * i});")
        i += 1
        t.sleep(1)
        if browser.execute_script("return document.body.scrollHeight;") < screen_height * i:
            break
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    articles_list = []
    # select all (div tag have box-category-item class) inside of a div have id load-list-news
    article_containers = soup.select("#load-list-news div.box-category-item")

    for article in article_containers:
        title_element = article.select_one('h3.box-title-text')
        link_title_element = article.select_one("a.box-category-link-title")
        abstract_element = article.select_one("p.box-category-sapo")
        img_element = article.select_one('img.box-category-avatar')
        time_element = None # not found in the page
        
        if title_element and abstract_element and img_element:
            # remove the '\n' at first of title
            title = title_element.text.strip()
            link = link_title_element.get('href')
            abstract = abstract_element.text
            imglink = img_element.get('src')
            time = None
            topic = url.split('/')[3]
            
            # Format the time into a readable format
            if time and len(time) >= 2:
                time_formatted = f"{time[0]} {time[1]}"
            else:
                time_formatted = "Unknown Time"

            # Append the article's information to the list
            articles_list.append({
                "title": title,
                "abstract": abstract,
                "source": "Báo Tuổi Trẻ",
                "link": f"https://tuoitre.vn{link}",
                "topic": dict_topic[topic],
                "time": time_formatted,
                "imglink": imglink
            })
    
    data4 = pd.concat([data4, pd.DataFrame(articles_list)], ignore_index=True)
    return data4

def find_all_articles_at_dt(maxsl, url, data4):
    dt_news_pages = []
    dt_news_pages.append(url+".htm")
    for i in range(2, maxsl):
        dt_news_pages.append(f"{url}/trang-{i}.htm")


    def get_dt_articles(web_url,data):
        response = requests.get(web_url,verify=False)
        soup = BeautifulSoup(response.content, 'html.parser')
        articles_list = []
        article_containers = soup.find_all("article", class_="article-item")

        for article in article_containers:
            # tag a have dt-text-black-mine class
            title_element = article.select_one('a.dt-text-black-mine')
            # inside div tag have class article-excerpt and inside an a tag
            abstract_element = article.select_one('div.article-excerpt a')
            # img tag have 2 class entered and loaded
            img_element = article.select_one('img')
            time_element = None
            
            if title_element and abstract_element and img_element:
                title = title_element.text
                link = title_element.get('href')
                abstract = abstract_element.text
                imglink = img_element.get('data-src')
                if(imglink == None): # if imglink['data-src'] is None, since src could be a data:base64 and not a link
                    continue
                time = None
                topic = url.split('/')[-1]
                
                # Format the time into a readable format
                if time and len(time) >= 2:
                    time_formatted = f"{time[0]} {time[1]}"
                else:
                    time_formatted = "Unknown Time"

                # Append the article's information to the list
                articles_list.append({
                    "title": title,
                    "abstract": abstract,
                    "source": "Báo Dân Trí",
                    "link": f"https://dantri.com.vn{link}",
                    "topic": dict_topic[topic],
                    "time": time_formatted,
                    "imglink": imglink
                })
    
        data = pd.concat([data, pd.DataFrame(articles_list)], ignore_index=True)
        return data
        
    for page in dt_news_pages:
        data4 = get_dt_articles(page, data4)

    return data4

def find_all_articles_at_vnexpress(maxsl, url, data4):
    vne_news_pages = []
    vne_news_pages.append(url)
    for i in range(2, maxsl):
        vne_news_pages.append(f"{url}-p{i}")

    def get_vne_articles(web_url, data):
        try:
            response = requests.get(web_url,verify=False)
        except:
            return data
        soup = BeautifulSoup(response.content, 'html.parser')
        articles_list = []
        article_containers = soup.find_all("article")

        for article in article_containers:
            # tag have class title-news
            title_element = article.select_one('h3.title-news a')
            abstract_element = article.select_one('p.description a')
            # img inside of article tag
            img_element = article.select_one('img')
            time_element = None
            
            if title_element and abstract_element and img_element:
                title = title_element.text.strip()
                link = title_element.get('href')
                abstract = abstract_element.text.strip()
                imglink = img_element.get('data-src')
                if(imglink == None): # if imglink['data-src'] is None, since src could be a data:base64 and not a link
                    continue
                time = None
                topic = url.split('/')[-1]
                
                # Format the time into a readable format
                if time and len(time) >= 2:
                    time_formatted = f"{time[0]} {time[1]}"
                else:
                    time_formatted = "Unknown Time"

                # Append the article's information to the list
                articles_list.append({
                    "title": title,
                    "abstract": abstract,
                    "source": "Báo VnExpress",
                    "link": link,
                    "topic": dict_topic[topic],
                    "time": time_formatted,
                    "imglink": imglink
                })
        
        return pd.concat([data, pd.DataFrame(articles_list)], ignore_index=True)

    for page in vne_news_pages:
        data4 = get_vne_articles(page, data4)
    
    return data4


In [16]:
for i in tqdm(range(0, len(vtc_news))):
    data4 = find_all_articles_at_vtc(10, vtc_news[i], data4)
for i in tqdm(range(0, len(ld_news))):
    data4 = find_all_articles_at_ld(10, ld_news[i], data4)
for i in tqdm(range(0, len(tt_news))):
    data4 = find_all_articles_at_tt(tt_news[i], data4)
for i in tqdm(range(0, len(dt_news))):
    data4 = find_all_articles_at_dt(10, dt_news[i], data4)
for i in tqdm(range(0, len(vne_news))):
    data4 = find_all_articles_at_vnexpress(10, vne_news[i], data4)

100%|██████████| 41/41 [04:17<00:00,  6.27s/it]


In [17]:
data4 = data4.drop_duplicates(subset = 'title', keep = 'last')
data4

,title,abstract,source,link,topic,time,imglink
0,EVNSPC: Tăng cường cảnh báo người dân sử dụng ...,Tổng công ty Điện lực miền Nam (EVNSPC) khuyến...,Báo VTC,https://vtcnews.vn/evnspc-tang-cuong-canh-bao-...,Thời sự,14/10/2024 15:28,https://cdn-i.vtcnews.vn/resize/me/upload/2024...
1,Tháp Thần Nông ở Bắc Ninh được xác lập Kỷ lục ...,Tháp Thần Nông - biểu tượng cho vị thần của nề...,Báo VTC,https://vtcnews.vn/thap-than-nong-o-bac-ninh-d...,Thời sự,14/10/2024 14:11,https://cdn-i.vtcnews.vn/resize/me/upload/2024...
2,Thị xã Thuận Thành: Hội nông dân trao quà cho ...,Hội nông dân thị xã Thuận Thành (Bắc Ninh) tổ ...,Báo VTC,https://vtcnews.vn/thi-xa-thuan-thanh-hoi-nong...,Thời sự,14/10/2024 14:04,https://cdn-i.vtcnews.vn/resize/me/upload/2024...
3,Quảng Ninh: Nhiệm vụ phòng cháy chữa cháy rừng...,"Sau bão Yagi, nguy cơ cháy rừng ở Quảng Ninh n...",Báo VTC,https://vtcnews.vn/quang-ninh-nhiem-vu-phong-c...,Thời sự,14/10/2024 12:15,https://cdn-i.vtcnews.vn/resize/me/upload/2024...
4,"Tổng Bí thư, Chủ tịch nước dự khai giảng lớp b...","Theo Tổng Bí thư, Chủ tịch nước, tổ chức lớp h...",Báo VTC,https://vtcnews.vn/tong-bi-thu-chu-tich-nuoc-d...,Thời sự,14/10/2024 11:39,https://cdn-i.vtcnews.vn/resize/me/upload/2024...
...,...,...,...,...,...,...,...
18141,Trao tặng cặp phao cứu sinh cho học sinh nghèo...,"Ngày 4.9, Ngân hàng Thương mại Cổ phần Ngoại t...",Báo Lao Động,https://laodong.vn/https://laodong.vn/tlv-tin-...,TLV tin hoạt động,04/09/2020 18:45,https://media-cdn-v2.laodong.vn/storage/newspo...
18142,Trao 500 cặp phao cứu sinh cho học sinh vùng s...,"Ngày 4.9, Ngân hàng Thương mại cổ phần Đầu tư ...",Báo Lao Động,https://laodong.vn/https://laodong.vn/tlv-tin-...,TLV tin hoạt động,04/09/2020 14:10,https://media-cdn-v2.laodong.vn/storage/newspo...
18143,Cặp phao Quỹ Tấm lòng vàng đến với học sinh ng...,"Trước thềm năm học mới 2020-2021, thông qua Qu...",Báo Lao Động,https://laodong.vn/https://laodong.vn/cong-doa...,TLV tin hoạt động,01/09/2020 16:05,https://media-cdn-v2.laodong.vn/storage/newspo...
18144,Kiên Giang: Đưa Học bổng Tấm lòng vàng về huyệ...,"Sau khi gửi lời chúc đến phụ huynh, các cháu đ...",Báo Lao Động,https://laodong.vn/https://laodong.vn/cong-doa...,TLV tin hoạt động,01/09/2020 16:03,https://media-cdn-v2.laodong.vn/storage/newspo...


In [19]:
data4.to_csv('vtc.csv', index = False, encoding = 'utf-8-sig')